In [1]:
import pandas as pd

In [2]:
data = pd.read_excel("./ie_data.xls", sheet_name="Data_2")[:-1]
# note to any economists out there: please do not store your dates in numeric YYYY.MM format
data["year"] = data["Date"].apply(round)
data["month"] = ((data["Date"] - data["year"]) * 100).apply(round)
data.set_index(["year", "month"], inplace=True)

sp500p_dict = data["SP500P"].to_dict()
dividend_dict = data["Dividend"].to_dict()
bond_return_dict = data["Returns"].to_dict()
cpi_dict = data["CPI"].to_dict()

In [3]:
def incr_year_month(year, month):
    if month == 12:
        return year + 1, 1
    else:
        return year, month + 1

def year_range(low_year, low_month, high_year, high_month):
    current_year = low_year
    current_month = low_month
    while current_year < high_year or (current_year == high_year and current_month < high_month):
        yield current_year, current_month
        current_year, current_month = incr_year_month(current_year, current_month)

In [4]:
# constant real income, constant savings rate
def contribute_constant_income_constant_rate(start_year, start_month, curr_year, curr_month, savings_rate, withdrawal_rate):
    inflation = (cpi_dict[curr_year, curr_month] / cpi_dict[start_year, start_month])
    income = 1 * inflation
    return income * savings_rate / 12, (1 - savings_rate) * income / withdrawal_rate

# 3% real raise, constant savings rate
def contribute_raise_constant_rate(start_year, start_month, curr_year, curr_month, savings_rate, withdrawal_rate):
    inflation = (cpi_dict[curr_year, curr_month] / cpi_dict[start_year, start_month])
    income = (1.03 ** (curr_year - start_year)) * inflation
    return income * savings_rate / 12, (1 - savings_rate) * income / withdrawal_rate

# 3% real raise, constant real spending
def contribute_raise_constant_spending(start_year, start_month, curr_year, curr_month, savings_rate, withdrawal_rate):
    inflation = (cpi_dict[curr_year, curr_month] / cpi_dict[start_year, start_month])
    income = (1.03 ** (curr_year - start_year)) * inflation
    spending = (1 - savings_rate) * inflation
    return (income - spending) / 12, spending / withdrawal_rate

In [5]:
withdrawal_rates = [.03, .035, .04, .045, .05, .055, .06]
savings_rates = [rate / 100 for rate in list(range(20, 90, 10))]
retirement_lengths = [30, 40, 50]
savings_profiles = [contribute_constant_income_constant_rate, contribute_raise_constant_rate, contribute_raise_constant_spending]
BOND_ALLOCATION = 0.25

In [6]:
import itertools

results = []
for withdrawal_rate, savings_rate, retirement_length, contribution_profile, bond_allocation in itertools.product(withdrawal_rates, savings_rates, retirement_lengths, savings_profiles, [BOND_ALLOCATION]):
    for start_year, start_month in year_range(1871, 1, 2025, 1):
        stock_quantity = 0
        bond_value = 0
        target_achieved = False
        for year, month in year_range(start_year, start_month, 2025, 1):
            inflation = cpi_dict[year, month] / cpi_dict[start_year, start_month]
            contribution, target = contribution_profile(start_year, start_month, year, month, savings_rate, withdrawal_rate)
            total_savings = (
                stock_quantity * sp500p_dict[year, month]
                + stock_quantity * dividend_dict[year, month] / 12
                + bond_value * bond_return_dict[year, month]
                + contribution
            )
            stock_quantity = total_savings * (1 - bond_allocation) / sp500p_dict[year, month]
            bond_value = total_savings * bond_allocation
            if total_savings >= target:
                target_achieved = True
                break
        results.append({
            "withdrawal_rate": withdrawal_rate,
            "savings_rate": savings_rate,
            "retirement_length": retirement_length,
            "contribution_profile": contribution_profile.__name__,
            "bond_allocation": bond_allocation,
            "start_year": start_year,
            "start_month": start_month,
            "end_year": year,
            "end_month": month,
            "target_achieved": target_achieved})


In [7]:
contribution_results_df = pd.DataFrame(results)

In [8]:
contribution_results_df

,withdrawal_rate,savings_rate,retirement_length,contribution_profile,bond_allocation,start_year,start_month,end_year,end_month,target_achieved
0,0.03,0.2,30,contribute_constant_income_constant_rate,0.25,1871,1,1901,6,True
1,0.03,0.2,30,contribute_constant_income_constant_rate,0.25,1871,2,1901,6,True
2,0.03,0.2,30,contribute_constant_income_constant_rate,0.25,1871,3,1901,6,True
3,0.03,0.2,30,contribute_constant_income_constant_rate,0.25,1871,4,1902,8,True
4,0.03,0.2,30,contribute_constant_income_constant_rate,0.25,1871,5,1904,11,True
...,...,...,...,...,...,...,...,...,...,...
814963,0.06,0.8,50,contribute_raise_constant_spending,0.25,2024,8,2024,12,False
814964,0.06,0.8,50,contribute_raise_constant_spending,0.25,2024,9,2024,12,False
814965,0.06,0.8,50,contribute_raise_constant_spending,0.25,2024,10,2024,12,False
814966,0.06,0.8,50,contribute_raise_constant_spending,0.25,2024,11,2024,12,False


In [9]:
import plotly.express as px

fig = px.histogram(
    contribution_results_df[(contribution_results_df["target_achieved"]) & (contribution_results_df["end_year"] >= 1926)],
    x="end_year",
    labels={"end_year": "Retirement Year", "contribution_profile": "Savings Method"},
    nbins=2025-1926,
    color="contribution_profile",
    barmode="overlay")
fig.show()

In [10]:
results = []
for withdrawal_rate, retirement_length, bond_allocation in itertools.product(withdrawal_rates, retirement_lengths, [BOND_ALLOCATION]):
    for start_year, start_month in year_range(1871, 1, 2025, 1):
        stock_quantity = (1 - bond_allocation) / sp500p_dict[start_year, start_month]
        bond_value = bond_allocation
        failed = False
        completed = False
        max_year = min(start_year + retirement_length, 2024)
        max_month = 12 if start_year + retirement_length >= 2025 else start_month
        upto_year, upto_month = incr_year_month(max_year, max_month)
        for year, month in year_range(start_year, start_month, upto_year, upto_month):
            inflation = cpi_dict[year, month] / cpi_dict[start_year, start_month]
            withdrawal = withdrawal_rate / 12 * inflation
            total_savings = (
                stock_quantity * sp500p_dict[year, month]
                + stock_quantity * dividend_dict[year, month] / 12
                + bond_value * bond_return_dict[year, month]
                - withdrawal
            )
            stock_quantity = total_savings * (1 - bond_allocation) / sp500p_dict[year, month]
            bond_value = total_savings * bond_allocation
            if total_savings <= 0:
                failed = True
                break
            if year == start_year + retirement_length and month == start_month:
                completed = True
        results.append({
            "withdrawal_rate": withdrawal_rate,
            "retirement_length": retirement_length,
            "bond_allocation": bond_allocation,
            "start_year": start_year,
            "start_month": start_month,
            "end_year": year,
            "end_month": month,
            "failed": failed,
            "completed": completed,
        })

In [11]:
retirement_results_df = pd.DataFrame(results)

In [12]:
fig = px.histogram(
    retirement_results_df[
        (retirement_results_df["start_year"] >= 1926)
        & (retirement_results_df["start_year"] + retirement_results_df["retirement_length"] <= 2025)
        & (retirement_results_df["completed"] | retirement_results_df["failed"])],
    x="start_year",
    labels={"start_year": "Retirement Year", "success": "Success"},
    nbins=2002-1926,
    color="failed")
fig.show()

In [13]:
uniform_survival = retirement_results_df[
    ((retirement_results_df["completed"]) | (retirement_results_df["failed"]))
    & (retirement_results_df["start_year"] >= 1926)
    & (retirement_results_df["start_year"] + retirement_results_df["retirement_length"] <= 2025)
].groupby(["retirement_length", "withdrawal_rate"]).agg({"failed": lambda df: 1 - df.sum() / df.count()})

In [14]:
uniform_survival

failed
retirement_length withdrawal_rate          
30                0.030            1.000000
                  0.035            1.000000
                  0.040            0.971014
                  0.045            0.891304
                  0.050            0.794686
                  0.055            0.708937
                  0.060            0.640097
40                0.030            1.000000
                  0.035            1.000000
                  0.040            0.912429
                  0.045            0.788136
                  0.050            0.661017
                  0.055            0.574859
                  0.060            0.494350
50                0.030            1.000000
                  0.035            0.986395
                  0.040            0.846939
                  0.045            0.673469
                  0.050            0.532313
                  0.055            0.408163
                  0.060            0.338435

In [15]:
retirement_date_counts = contribution_results_df[
    (contribution_results_df["end_year"] >= 1926)
    & (contribution_results_df["end_year"] < 1995)
    & (contribution_results_df["target_achieved"])
].groupby(["end_year", "end_month"]).size()

In [18]:
retirement_date_counts.index.rename({"end_year": "start_year", "end_month": "start_month"}, inplace=True)
retirement_date_counts.name = "count"

In [19]:
retirement_results_df[
        (retirement_results_df["start_year"] >= 1926)
        & (retirement_results_df["start_year"] + retirement_results_df["retirement_length"] <= 2025)
        & (retirement_results_df["completed"] | retirement_results_df["failed"])
].merge(
    retirement_date_counts, on=["start_year", "start_month"]
).groupby(
    ["retirement_length", "withdrawal_rate"]
).apply(lambda df: 1 - (df[df["failed"]]["count"].sum() / df["count"].sum())).reset_index()

/tmp/ipykernel_30172/860008341.py:9: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,retirement_length,withdrawal_rate,0
0,30,0.030,1.000000
1,30,0.035,1.000000
2,30,0.040,0.963284
3,30,0.045,0.888890
4,30,0.050,0.802795
5,30,0.055,0.713551
6,30,0.060,0.656623
7,40,0.030,1.000000
8,40,0.035,1.000000
9,40,0.040,0.909524
